In [168]:
import numpy as np
import pandas as pd
import json

In [169]:
from datetime import date

today = date.today()
print("Today's date:", today)

Today's date: 2022-10-27


In [170]:
import http.client, urllib.parse
ACCESS_KEY = '243247ebaf607124b6b062d06e319452'

conn = http.client.HTTPConnection('api.mediastack.com')

params = urllib.parse.urlencode({
    'access_key': ACCESS_KEY,
    'languages' : 'en',
    'limit' : '100',
    'date' : today,
    'categories' : 'business'
    })

conn.request('GET', '/v1/news?{}'.format(params))

res = conn.getresponse()
data = res.read()

print(data.decode('utf-8'))

{"pagination":{"limit":100,"offset":0,"count":100,"total":298},"data":[{"author":"GlobeNewswire","title":"Imperial Provides Production and Red Chris Exploration Update","description":"VANCOUVER, British Columbia, Oct. 26, 2022 (GLOBE NEWSWIRE) \u0026#8212; Imperial Metals Corporation (the \u201cCompany\u201d) (TSX:III) reports Red Chris metal production (100%) for the third quarter of 2022 was 19.2 million pounds copper and 17,513 ounces gold, down 13% and 10% respectively from the 22.0 million pounds copper and 19,540 ounces gold produced during the second [\u0026#8230;]","url":"https:\/\/financialpost.com\/globe-newswire\/imperial-provides-production-and-red-chris-exploration-update","source":"Financial Post | Canada Business News","image":null,"category":"business","language":"en","country":"us","published_at":"2022-10-27T01:03:30+00:00"},{"author":"Yongchang Chin","title":"Oil Advances For Third Day On Record US Exports, Weaker Dollar","description":"Oil Advances For Third Day On R

In [171]:
dict_data = json.loads(data.decode('utf-8'))

In [172]:
print(dict_data['data'][0])

{'author': 'GlobeNewswire', 'title': 'Imperial Provides Production and Red Chris Exploration Update', 'description': 'VANCOUVER, British Columbia, Oct. 26, 2022 (GLOBE NEWSWIRE) &#8212; Imperial Metals Corporation (the “Company”) (TSX:III) reports Red Chris metal production (100%) for the third quarter of 2022 was 19.2 million pounds copper and 17,513 ounces gold, down 13% and 10% respectively from the 22.0 million pounds copper and 19,540 ounces gold produced during the second [&#8230;]', 'url': 'https://financialpost.com/globe-newswire/imperial-provides-production-and-red-chris-exploration-update', 'source': 'Financial Post | Canada Business News', 'image': None, 'category': 'business', 'language': 'en', 'country': 'us', 'published_at': '2022-10-27T01:03:30+00:00'}


In [173]:
# for x in dict_data['data']:
#     df = pd.DataFrame.from_dict(x, orient='index',
#                        columns=['author', 'title', 'description', 'url','source','image','category','language','country','published_at'])

In [174]:
df = pd.DataFrame()

In [175]:
type(df)

pandas.core.frame.DataFrame

In [176]:
 for x in dict_data['data']:
    df = df.append(x, ignore_index=True)

C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868

C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868

C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868\1018869995.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(x, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_18868

In [177]:
df

,author,title,description,url,source,image,category,language,country,published_at
0,GlobeNewswire,Imperial Provides Production and Red Chris Exp...,"VANCOUVER, British Columbia, Oct. 26, 2022 (GL...",https://financialpost.com/globe-newswire/imper...,Financial Post | Canada Business News,None,business,en,us,2022-10-27T01:03:30+00:00
1,Yongchang Chin,Oil Advances For Third Day On Record US Export...,Oil Advances For Third Day On Record US Export...,https://www.bqprime.com/markets/oil-advances-f...,Bloomberg | Latest And Live Business,None,business,en,us,2022-10-27T01:05:41+00:00
2,Alex Barinka,Mark Zuckerberg Asks For ‘Patience’ As Meta’s ...,Mark Zuckerberg Asks For ‘Patience’ As Meta’s ...,https://www.bqprime.com/business/meta-s-zucker...,Bloomberg | Latest And Live Business,None,business,en,us,2022-10-27T04:28:35+00:00
3,None,"Gland Pharma plunges 13%, hits 52-week low on ...","Earnings before interest, taxes, depreciation,...",https://www.business-standard.com/article/mark...,Business Standard,None,business,en,us,2022-10-27T04:25:32+00:00
4,None,How a Mumbai-based startup is making life easy...,"GetVantage, a start-up offering revenue financ...",https://www.businesstoday.in/latest/corporate/...,Business Today | Latest Stock Market And Econo...,None,business,en,in,2022-10-27T05:16:59+00:00
...,...,...,...,...,...,...,...,...,...,...
95,zee business,HDFC Bank FD Interest Rates 2022 hiked again -...,HDFC Bank FD Interest Rates 2022: The hiked FD...,https://www.zeebiz.com/personal-finance/news-h...,Zee Business,None,business,en,us,2022-10-27T08:55:45+00:00
96,None,Newcrest Mining says quarterly gold production...,Newcrest Mining says quarterly gold production...,https://seekingalpha.com/news/3896097-newcrest...,Seeking Alpha,None,business,en,us,2022-10-27T00:10:43+00:00
97,Reuters,"ECB to hike interest rates, likely to trim ban...",FRANKFURT &#8212; The European Central Bank wi...,https://financialpost.com/pmn/business-pmn/ecb...,Financial Post | Canada Business News,None,business,en,us,2022-10-27T00:45:14+00:00
98,Peter Lewis,I was a Medibank ‘Private’ customer. Let me te...,Citizens must reclaim control of their identit...,https://www.theage.com.au/business/companies/i...,The Age,https://static.ffx.io/images/$zoom_0.2043%2C$m...,business,en,au,2022-10-27T00:58:59+00:00


In [178]:
title_df = df['title']

In [179]:
title_df

0     Imperial Provides Production and Red Chris Exp...
1     Oil Advances For Third Day On Record US Export...
2     Mark Zuckerberg Asks For ‘Patience’ As Meta’s ...
3     Gland Pharma plunges 13%, hits 52-week low on ...
4     How a Mumbai-based startup is making life easy...
                            ...                        
95    HDFC Bank FD Interest Rates 2022 hiked again -...
96    Newcrest Mining says quarterly gold production...
97    ECB to hike interest rates, likely to trim ban...
98    I was a Medibank ‘Private’ customer. Let me te...
99    Forrest makes green plea as Fortescue delivers...
Name: title, Length: 100, dtype: object